In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
from response.feature import Feature

In [3]:
import os

def json2data(path):
    cols = ["text", "label", "subLabel"]
    df = pd.DataFrame(index=[], columns=cols)
    files = os.listdir(path)
    for cop in files:
        if "." not in cop:
            continue
        with open(path+cop, "r") as f:
            json_data = json.load(f)
            mode = cop.split(".")[0]
            max_ = 300
            for i, data in enumerate( json_data[mode] ) :
                if i > max_:
                    break
                text = data["data"]
                label = data["label"][0]
                subLabel = ""
                df = df.append(pd.DataFrame([text, label, subLabel], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [4]:
label_list = "YN WH please proposal plain".split()
label_dict = dict( zip(label_list, range(len(label_list))) )

def extract_X_y(df):
    X = []
    y = []
    for te, la in zip(df.text, df.label):
        X.append(te)
        y.append(label_dict[la])
    return X, y

In [5]:
corpus_root = "../../corpus"
# name = "question/short"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"
data_path

'../../corpus/question/'

In [6]:
df = json2data(data_path)
df

,text,label,subLabel
0,メニューを見せていただけますか？,please,
1,おいでいただけますか？,please,
2,マッシュポテトをもらえますか？,please,
3,伝言を預かっていただけますか？,please,
4,ご一緒しませんか？,please,
...,...,...,...
1236,メンバーになりたいかい？,YN,
1237,明日スタートなさいますか？,YN,
1238,トレイがいりますか？,YN,
1239,2階も見たいですか？,YN,


In [7]:
df["label"].value_counts()

proposal    301
plain       301
YN          291
please      241
WH          107
Name: label, dtype: int64

In [8]:
X, y = extract_X_y(df)

In [9]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [10]:
f = Feature()

In [11]:
F = Feature()
F_path = "../X_y_data/response2/"
F_name = "Classify_F.pickle"
featureM = DataManager(F_path)
F.make_features(X_train_str)

In [ ]:
featureM.save_data(F_name, F)

In [12]:
print(F.feature_num)
X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

20960


In [13]:
lr = LogisticRegression(solver='sag', max_iter=10000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[66  7  9  7  2]
 [ 7 17  1  6  0]
 [ 6  0 48  3  3]
 [ 6  1  4 88  2]
 [ 0  0  0  1 89]]
accuracy =  0.8257372654155496


In [15]:
import ginza
for i, token in enumerate(doc):
    # print(token.orth_, token.pos_, ginza.inflection(token))
    if token.pos_=="VERB" or "AUX":
        inflection = ginza.inflection(token)
        if inflection=="":
            continue
        conj = ginza.inflection(token).split(",")[1].split("-")[0]
        if conj=="連用形" and i< (len(doc)-1):
            print(token)
            if doc[i+1].orth_=="て": 
                print(token, doc[i+1], min(3, len(doc)-1-i))
                f = "-".join( [token.orth_ for token in doc[i:i+min(3, len(doc)-1-i)] ] )
                print(f)   
    else:
        print("")

NameError: name 'doc' is not defined

In [ ]:
model_path = "../models/response2/"
model_name = "lassify_M.pickle"
modelM = DataManager(model_path)
print(model_name)

In [ ]:
modelM.save_data(model_name, lr)

In [4]:
2060000/(3600*41)

13.956639566395664

In [3]:
5000/120

41.666666666666664